[pybox2d 2.1.0 manual](https://github.com/pybox2d/pybox2d/wiki/manual)

[Microsoft Visual C++ Compiler for Python 2.7](https://www.microsoft.com/en-us/download/confirmation.aspx?id=44266)

# System Requirements

- python 2.7.6 `python -V`
- Pygame 1.9.3 `pip freeze | grep pygame`
- swig [swig.exe download](http://www.swig.org/download.html)

```
sudo apt-get install python
sudo pip install pygame
sudo apt-get install swig
```

1.

```
git clone https://github.com/pybox2d/pybox2d.git
python setup.py build
sudo python setup.py install
```

2.

```
sudo pip install box2d
```

3.

[pybox2d downloads](https://code.google.com/archive/p/pybox2d/downloads)

In [1]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import division
from Box2D import *
from pygame.locals import (QUIT, KEYDOWN, K_ESCAPE)
import math
import random
import numpy as np
import pygame as pg
import matplotlib.pyplot as plt

def plot():
    plt.plot(x, y, marker='*')
    for m in modules:
        plt.plot([m.position[0]], [m.position[1]], marker='o')
    plt.show()


def world2screen(v):
    world = np.array((max(x) - min(x), max(y) - min(y)))
    screen = np.array(SCREEN)
    scale = min(np.divide(screen, world))
    offset = screen / 2 - scale * np.array((min(x) + max(x), min(y) + max(y))) / 2
    r = scale * np.array(v) + offset
    r[1] = SCREEN[1] - r[1]
    return r

In [2]:
N = 3
POLYGON = '../tr1_1.csv'
lam = 0
TARGET_FPS = 24
TIME_STEP = 1 / TARGET_FPS
SCREEN = (640, 480)
random.seed(0)

In [3]:
from ITPLA import *

In [4]:
# read the polygon from .csv file with normalizing and scaling
polygon = np.array(
    map(lambda p: map(eval, p.split()), open(POLYGON))) / 100 #* 1.5
polygon = (polygon - polygon.mean(axis=0))
polygon = polygon[::-1]
x, y = zip(*polygon)
print area(*zip(*polygon))
# This defines a triangle in CCW order.

module = np.array(
    map(lambda a: [math.cos(a), math.sin(a)], 2 * math.pi / N * np.arange(N)))
print area(*zip(*module))
B_P = int(area(*zip(*polygon)) / area(*zip(*module)))

5.17306601162
1.29903810568


In [5]:
def my_draw_polygon(self, body):
    pg.draw.lines(screen, (255, 0, 0, 0), True, map(lambda v: world2screen(body.transform * v), self.vertices))
    for c in filter(lambda c: c.contact.touching, body.contacts):
        vertices = [world2screen(v) for v in [body.position, c.other.position]]
        pg.draw.lines(screen, (0, 0, 255, 0), False, vertices)
        A = c.contact.fixtureA
        B = c.contact.fixtureB
        res = b2Distance(shapeA=A.shape, shapeB=B.shape,
                         transformA=A.body.transform, transformB=B.body.transform)

        def f2i(p): return map(int, world2screen(p))
        pg.draw.circle(screen, (0, 255, 0, 0), f2i(res.pointA), 5)
        pg.draw.circle(screen, (0, 255, 0, 0), f2i(res.pointB), 5)


def my_draw_edge(self, body):
    pg.draw.lines(screen, (0, 0, 0, 0), True, map(lambda v: world2screen(body.transform * v), self.vertices))


b2.polygonShape.draw = my_draw_polygon
b2.edgeShape.draw = my_draw_edge

In [6]:
screen = pg.display.set_mode(SCREEN, 0, 32)
pg.display.set_caption('ITPLA')
clock = pg.time.Clock()

for w in ITPLA(polygon, module, B_P):
    if any(map(lambda e: e.type == QUIT or (e.type == KEYDOWN and e.key == K_ESCAPE), pg.event.get())):
        pg.quit()
        break
    screen.fill((220, 220, 220, 0))
    for b in w.bodies:
        b.fixtures[0].shape.draw(b)
    pg.display.flip()
    clock.tick()  # TARGET_FPS)
    pg.display.set_caption('ITPLA {}'.format(clock.get_fps()))